## HTTP Header Authentication for MCP Servers in APIM

In [ ]:
%pip install agent-framework==1.0.0b251001 python-dotenv azure-ai-projects==1.1.0b4

### Setting Up the Environment

In [ ]:
import os
from dotenv import load_dotenv
from azure.ai.agents.models import (
    ListSortOrder,
    McpTool,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
)
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

load_dotenv()
project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
model = os.getenv("AI_FOUNDRY_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

### Creating our MCP Tool Object

In [ ]:
mcp_tool = McpTool(
    server_label = "demo",
    server_url = "https://kuljot-apim-standard.azure-api.net/test-mcp-server/mcp",
)



### Creating our MCP Agent

In [ ]:
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

agents_client = project_client.agents

agent = agents_client.create_agent(
        model=model,
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )

print(f"Created agent, ID: {agent.id}")

### Creating Thread and Adding Auth Header to MCP Tool

In [ ]:
# Create thread for communication
thread = agents_client.threads.create()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="tell me about courses with rating greater than 4....which one has the maximum lectures?",
    )
print(f"Created message, ID: {message.id}")

# Create and process agent run in thread with MCP tools
mcp_tool.update_headers("Authorization", "kuljot123")
mcp_tool.set_approval_mode("never")

### Running our Agent Thread

In [ ]:
run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
print(f"Created run, ID: {run.id}")

while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

print(f"Run completed with status: {run.status}")
if run.status == "failed":
        print(f"Run failed: {run.last_error}")

### Print All the Messages in the Thread

In [ ]:
# Fetch and log all messages
messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
print("\nConversation:")
print("-" * 50)
for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)